In [ ]:
import requests
import xmltodict
import json
from pymongo import MongoClient
import time

# ✅ MongoDB 연결
client = MongoClient('mongodb://localhost:27017')
db = client['pharmacy_db']
collection = db['pharmacy_locations']

# ✅ 공공데이터포털 인증키 (Encoding된 형태 사용)
SERVICE_KEY = "PYgq7s18hS5yFhC/5Azhj6zMKVBknzBEZKJdcyOX/LA7d18yE+5LhuLLdE7ZDtWRgIjdwNQJBnyheI4HSem/qg=="

# ✅ 요청 URL
url = "http://apis.data.go.kr/B551182/pharmacyInfoService/getParmacyBasisList"


# ✅ 여러 페이지 조회
for page in range(1, 6):  # 예시: 1~5페이지만 가져옴. 데이터 많으면 늘려도 됨
    params = {
        'serviceKey': SERVICE_KEY,
        'pageNo': page,
        'numOfRows': 100,
    }

    res = requests.get(url, params=params)
    data_dict = xmltodict.parse(res.content)
    items = data_dict.get('response', {}).get('body', {}).get('items', {}).get('item', [])

    # 하나일 경우 dict로 오기 때문에 리스트로 감싸줌
    if isinstance(items, dict):
        items = [items]

    # ✅ MongoDB에 저장
    for item in items:
        doc = {
            '약국명': item.get('yadmNm'),
            '주소': item.get('addr'),
            '전화번호': item.get('telno'),
            '위도': item.get('YPos'),
            '경도': item.get('XPos'),
        }
        collection.insert_one(doc)

    print(f"✅ {page}페이지 저장 완료")
    time.sleep(1)  # API 요청 속도 제한 방지

print("🎉 전체 저장 완료")

✅ 1페이지 저장 완료
✅ 2페이지 저장 완료
✅ 3페이지 저장 완료
✅ 4페이지 저장 완료
✅ 5페이지 저장 완료
🎉 전체 저장 완료


In [ ]:
import requests
import xmltodict
import time
from pymongo import MongoClient

# MongoDB 연결
client = MongoClient("mongodb://localhost:27017")
db = client["pharmacy_db"]
collection = db["pharmacy_info"]

# 공공데이터 인증키 & API
SERVICE_KEY = "PYgq7s18hS5yFhC/5Azhj6zMKVBknzBEZKJdcyOX/LA7d18yE+5LhuLLdE7ZDtWRgIjdwNQJBnyheI4HSem/qg=="
url = "http://apis.data.go.kr/B551182/pharmacyInfoService/getParmacyBasisList"

# ✅ 전체 데이터 개수 구하기
init_params = {
    'serviceKey': SERVICE_KEY,
    'pageNo': 1,
    'numOfRows': 1,
}
init_res = requests.get(url, params=init_params)
init_data = xmltodict.parse(init_res.content)
total = int(init_data['response']['body']['totalCount'])
pages = (total // 100) + 1

print(f"총 데이터 수: {total}개 / 총 페이지 수: {pages}페이지")

# ✅ 전체 페이지 반복
for page in range(1, pages + 1):
    print(f"📦 {page}/{pages} 페이지 처리 중...")
    params = {
        'serviceKey': SERVICE_KEY,
        'pageNo': page,
        'numOfRows': 100,
    }
    res = requests.get(url, params=params)
    data_dict = xmltodict.parse(res.content)
    items = data_dict.get('response', {}).get('body', {}).get('items', {}).get('item', [])

    if isinstance(items, dict):  # 단일 항목일 경우 리스트로 감싸기
        items = [items]

    for item in items:
        doc = {
            "place_name": item.get("yadmNm"),
            "road_address_name": item.get("addr"),
            "phone": item.get("telno"),
            "x": item.get("XPos"),
            "y": item.get("YPos"),
        }
        collection.insert_one(doc)
    time.sleep(1)  # 과도한 요청 방지

print("🎉 모든 데이터 저장 완료!")